In [ ]:
import os
import sys
from scgt import GeoTiff
sys.path.append('/Users/nvalett/Documents/Natalie/Species Dist Research/Code/ecoscape-earth/')
sys.path.append('/Users/nvalett/Documents/Natalie/Species Dist Research/Code/ecoscape-earth/ecoscape-connectivity/')
from EcoLinker.EcoLinker.restorationOptimizer import restorationOptimizer
from ecoscape_connectivity_local import repopulation, util
# import ecoscape_connectivity
from scgt import Tile, GeoTiff
import numpy as np

DATA_PATH="/Users/nvalett/Documents/Natalie/Species Dist Research/Thesis/EcoLinker/tests/assets"
HABITAT_PATH = os.path.join(DATA_PATH, "smol/hab_smol.tif")
TERRAIN_PATH = os.path.join(DATA_PATH, "smol/terrain.tif")
RESTORED_TERRAIN_PATH = os.path.join(DATA_PATH, "smol/restored_ter_smol.tif")
PERMEABILITY_PATH = os.path.join(DATA_PATH, "paper_repro/transmission_refined_1.csv")
transmission_d = util.read_transmission_csv(PERMEABILITY_PATH)

CONNECTIVITY_PATH = os.path.join(DATA_PATH, "smol/repop.tif")
RESTORED_CONNECTIVITY_PATH = os.path.join(DATA_PATH, "smol/repop_after_restoration.tif")
FLOW_PATH = os.path.join(DATA_PATH, "smol/grad.tif")
RESTORED_FLOW_PATH = os.path.join(DATA_PATH, "smol/grad_after_restoration.tif")
DEATH_PATH = os.path.join(DATA_PATH, "smol/death.tif")

optimizer = restorationOptimizer(HABITAT_PATH, TERRAIN_PATH, RESTORED_TERRAIN_PATH, CONNECTIVITY_PATH, FLOW_PATH, RESTORED_CONNECTIVITY_PATH, RESTORED_FLOW_PATH, DEATH_PATH, PERMEABILITY_PATH, 10)

Use same seeds

In [ ]:
# compute connectivity - not needed unless output is needed
import torch
hab = GeoTiff.from_file(optimizer.habitat_fn)
hab_tile = hab.get_all_as_tile().m.astype(np.float)
seeds = (torch.rand(400,*(hab_tile.shape)) < 0.002) * hab_tile # 1% of pixels are seeds.


repopulation.compute_connectivity(HABITAT_PATH, TERRAIN_PATH, CONNECTIVITY_PATH, FLOW_PATH, transmission_d, single_tile=True, seeds=seeds)
print(optimizer.sum_of_tif(CONNECTIVITY_PATH))

In [ ]:
repopulation.compute_connectivity(HABITAT_PATH, TERRAIN_PATH, CONNECTIVITY_PATH, FLOW_PATH, transmission_d, single_tile=True)
print(optimizer.sum_of_tif(CONNECTIVITY_PATH))

Restoration based on highest death rates:
1. Compute death layer by taking (flow/p) * (1-p)
2. Find top n pixels with highest death rates
3. Convert pixels to more permiable terrain

In [ ]:
death_tif = optimizer.get_death_layer(optimizer.death_fn)
highest_death = optimizer.get_highest_death_pixels(death_tif)
pre_restoration_conn = optimizer.sum_of_tif(CONNECTIVITY_PATH)

print(highest_death)

print(optimizer.get_most_permiable_terrain())

print('Restoration:')
print(f'Pre-restoration sum of connectivity {pre_restoration_conn}')

ter = GeoTiff.from_file(optimizer.terrain_fn)

# restore pixels with highest death rates
for x, y in highest_death.keys():
    with GeoTiff.from_file(TERRAIN_PATH) as terrain_geotiff:
        old_permiability = terrain_geotiff.get_pixel_value(x, y)

    optimizer.change_terrain(x, y)

    with GeoTiff.from_file(RESTORED_TERRAIN_PATH) as terrain_geotiff:
        new_permiability = terrain_geotiff.get_pixel_value(x, y)
    print(f'Restoring pixel ({x}, {y}) from permiability {optimizer.permeability_dict[old_permiability]} to {optimizer.permeability_dict[new_permiability]}')

pre_restoration_conn = optimizer.sum_of_tif(CONNECTIVITY_PATH)
optimizer.restore_pixels()

Compare new (post-restoration) connectivity to old connectivity
    - Use same seeds to enforce determinism

In [ ]:
# compute connectivity - not needed unless output is needed
repopulation.compute_connectivity(HABITAT_PATH, RESTORED_TERRAIN_PATH, RESTORED_CONNECTIVITY_PATH, RESTORED_FLOW_PATH, transmission_d, single_tile=True, seeds=seeds)
post_restoration_conn = optimizer.sum_of_tif(RESTORED_CONNECTIVITY_PATH)

# change in connectivity:
delta_conn = int(post_restoration_conn) - int(pre_restoration_conn)
percent_change = delta_conn / pre_restoration_conn

print(f'pre-restoration sum of connectivity {pre_restoration_conn}')
print(f'post-restoration sum of connectivity {post_restoration_conn}')
print(f'change in connectivity: {delta_conn}, or {percent_change.round(7)}%')

Try to compute regional connectivity gain for each restored pixel...

In [ ]:
PRE_TER = os.path.join(DATA_PATH, "smol/pre_ter.tif")
PRE_HAB = os.path.join(DATA_PATH, "smol/pre_hab.tif")
PRE_CONN = os.path.join(DATA_PATH, "smol/pre_conn.tif")
PRE_FLOW = os.path.join(DATA_PATH, "smol/pre_flow.tif")
POST_TER = os.path.join(DATA_PATH, "smol/post_ter.tif")
POST_HAB = os.path.join(DATA_PATH, "smol/post_hab.tif")
POST_CONN = os.path.join(DATA_PATH, "smol/post_conn.tif")
POST_FLOW = os.path.join(DATA_PATH, "smol/post_flow.tif")

In [ ]:
import rasterio
import torch

death_tif = optimizer.get_death_layer(optimizer.death_fn)
highest_death = optimizer.get_highest_death_pixels(death_tif, 2)

print(highest_death)
pre_restoration_conn = optimizer.sum_of_tif(CONNECTIVITY_PATH)

ter = GeoTiff.from_file(optimizer.terrain_fn)
hab = GeoTiff.from_file(optimizer.habitat_fn)

window_sz = 32

for x, y in highest_death.keys():
    lat_max, lon_max = rasterio.transform.xy(ter.dataset.transform, y-window_sz, x-window_sz)
    lat_min, lon_min = rasterio.transform.xy(ter.dataset.transform, y+window_sz, x+window_sz)

    pre_ter = ter.crop_to_new_file(PRE_TER, [lat_max, lon_min, lat_min, lon_max], padding=0)
    pre_ter.dataset.close()
    pre_hab = hab.crop_to_new_file(PRE_HAB, [lat_max, lon_min, lat_min, lon_max], padding=0)
    hab_tile = hab.get_all_as_tile().m.astype(np.float)
    pre_hab.dataset.close()

    seeds = (torch.rand(400,*(hab_tile.shape)) < 0.002) * hab_tile # 1% of pixels are seeds.
    repopulation.compute_connectivity(PRE_HAB, PRE_TER, PRE_CONN, PRE_FLOW, transmission_d, single_tile=True, seeds=seeds)

    pre_hab = GeoTiff.from_file(PRE_HAB)
    pre_hab.draw_geotiff()
    conn = GeoTiff.from_file(PRE_CONN)
    conn.draw_geotiff()
    # flow = GeoTiff.from_file(PRE_FLOW)
    # flow.draw_geotiff()

    pre_conn_sum = optimizer.sum_of_tif(PRE_CONN)

    post_ter = ter.crop_to_new_file(POST_TER, [lat_max, lon_min, lat_min, lon_max], padding=0)
    optimizer.change_terrain(x, y, POST_TER)

    repopulation.compute_connectivity(PRE_HAB, POST_TER, POST_CONN, POST_FLOW, transmission_d, single_tile=True, seeds=seeds)
    conn = GeoTiff.from_file(POST_CONN)
    conn.draw_geotiff()
    post_conn_sum = optimizer.sum_of_tif(POST_CONN)
    
    # change in connectivity:
    delta_conn = int(post_conn_sum) - int(pre_conn_sum)
    percent_change = delta_conn / pre_conn_sum

    print(f'pre-restoration sum of connectivity {pre_conn_sum}')
    print(f'post-restoration sum of connectivity {post_conn_sum}')
    print(f'change in connectivity: {delta_conn}, or {percent_change.round(7)}%')